In [1]:
# ghost schedule plot dump

In [ ]:
def get_frames(input_df, train_dict, train_x, train_y, train_text,
               exp_df, exp_dict, exp_x, exp_y, exp_text,
               time_col='time', exp_col='expected'): 
    # add exp columns
    min_time = min([input_df[time_col].min(), exp_df[exp_col].min()])
    current_time = min_time
    max_time = max([input_df[time_col].max(), exp_df[exp_col].max()])
    time_step = 60 # seconds
    slider_time_step = 900
    frames = []

    sliders_dict = {
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Time:',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 1000, 'easing': 'linear'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': []
    }
    
    def calculate_frames(train_dict, train_x, train_y):
        for t_id, train in train_dict.iteritems():
            time_cursor = current_time
            idx = train['idx']
            if not 'completed' in train:
                current_state = train['state_count']
                x_delta = 0
                y_delta = 0
                t = 0
                while time_cursor < animate_to:
                    state = train['state'][current_state]
                    if state['end'] > animate_to:
                        t = animate_to - time_cursor
                    else:
                        t = state['end'] - time_cursor
                        train['state_count'] = train['state_count'] + 1
                        if current_state < (len(train['state']) - 1):
                            current_state = current_state + 1
                        else:
    #                         t = t + (animate_to-time_cursor)
                            time_cursor = animate_to
                            train['completed'] = True
                    x_delta = x_delta + state['x_delta']*t.total_seconds()
                    y_delta = y_delta + state['y_delta']*t.total_seconds()
                    time_cursor = time_cursor + t
                train_x[idx] = train_x[idx] + x_delta
                train_y[idx] = train_y[idx] + y_delta
        return train_dict, train_x, train_y
    
    times = []
    total_time_step = 0

    while current_time < max_time:
        times.append(current_time)
        animate_to = current_time + dt.timedelta(seconds=time_step)
        
        train_dict, train_x, train_y = calculate_frames(train_dict, train_x, train_y)
        exp_dict, exp_x, exp_y = calculate_frames(exp_dict, exp_x, exp_y)
        
        slider_step = {'args': [
            [current_time],
            {'frame': {'duration': 700, 'redraw': False},
             'mode': 'immediate',
           'transition': {'duration': 1000, 'easing': 'linear'}}
         ],
         'label': current_time,
         'method': 'animate'}

        current_time = animate_to
        layout = {'annotations': [{'x': 10, 'y': 5, 'text': 'time: ' + current_time.strftime("%I:%M %p")}],
                 }
        frames.append({'data': [{'x': list(train_x), 'y': list(train_y), 'text': train_text}, 
                               {'x': list(exp_x), 'y': list(exp_y), 'text': exp_text}], 
                       'name': str(current_time),
                       'layout': layout})

        total_time_step = total_time_step + time_step
        if not (total_time_step % slider_time_step):
            sliders_dict['steps'].append(slider_step)
    return frames, sliders_dict, times

In [ ]:
def plot(frames, sliders_dict, times, title):
    
    figure = {'data': [go.Scatter({'x': [0, 1], 'y': [0, 1], 'type': 'scatter', 'mode':'markers', 'name': 'observed'}),
                       go.Scatter({'x': [0, 1], 'y': [0, 1], 'type': 'scatter', 'mode':'markers', 'marker':{'color':'rgba(152, 0, 0, .2)'}, 'name':'scheduled'})],

              'layout': {'xaxis': {'range': [1, 16], 
                                   'autorange': False, 
                                   'scaleanchor': "y",
                                   'ticks':'', 
                                   'showticklabels':False,
                                   'showgrid':False,
                                   'zeroline':False,
                                   'showline':False},
                         'yaxis': {'range': [-6, 6], 
                                   'autorange': False, 
                                   'ticks':'', 
                                   'showticklabels':False,
                                   'showgrid':False,
                                   'zeroline':False,
                                   'showline':False},
                         'title': title,
                         'width': 900,
                         'height': 600,
                         'annotations': [{
                                            'x':10,
                                            'y':0.5,
                                            'xref':'x',
                                            'yref':'y',
                                            'text':'SEC',
                                            'showarrow': False
                                         },
                                         {
                                            'x':2,
                                            'y':0.5,
                                            'xref':'x',
                                            'yref':'y',
                                             'ax':-30,
                                             'ay':0,
                                            'text':'',
                                            'showarrow': True,
                                            'arrowwidth':1,
                                         },
                                         {
                                            'x':1,
                                            'y':-0.5,
                                            'xref':'x',
                                            'yref':'y',
                                             'ax':30,
                                             'ay':0,
                                            'text':'',
                                            'showarrow': True,
                                            'arrowwidth':1,
                                         },
                                         {
                                            'x':14,
                                            'y':0.5,
                                            'xref':'x',
                                            'yref':'y',
                                             'ax':-30,
                                             'ay':0,
                                            'text':'',
                                            'showarrow': True,
                                            'arrowwidth':1,
                                         },
                                         {
                                            'x':13,
                                            'y':-0.5,
                                            'xref':'x',
                                            'yref':'y',
                                             'ax':30,
                                             'ay':0,
                                            'text':'',
                                            'showarrow': True,
                                            'arrowwidth':1,
                                         },
                                         {
                                            'x':6,
                                            'y':3,
                                            'xref':'x',
                                            'yref':'y',
                                             'ax':-8.6,
                                             'ay':-28.7,
                                            'text':'',
                                            'showarrow': True,
                                            'arrowwidth':1,
                                         },
                                         {
                                            'x':4.9,
                                            'y':3.8,
                                            'xref':'x',
                                            'yref':'y',
                                             'ax':8.6,
                                             'ay':28.7,
                                            'text':'',
                                            'showarrow': True,
                                            'arrowwidth':1,
                                         },
                                         {
                                            'x':9.9,
                                            'y':-3.8,
                                            'xref':'x',
                                            'yref':'y',
                                             'ax':-11.1,
                                             'ay':-27.8,
                                            'text':'',
                                            'showarrow': True,
                                            'arrowwidth':1,
                                         },
                                         {
                                            'x':8.8,
                                            'y':-3,
                                            'xref':'x',
                                            'yref':'y',
                                             'ax':11.1,
                                             'ay':27.78,
                                            'text':'',
                                            'showarrow': True,
                                            'arrowwidth':1,
                                         },
                                         #STATIONS
                                         {
                                            'x':10,
                                            'y':0.5,
                                            'xref':'x',
                                            'yref':'y',
                                            'text':'SEC',
                                            'showarrow': False
                                         },
                                         {
                                            'x':5,
                                            'y':0.5,
                                            'xref':'x',
                                            'yref':'y',
                                            'text':'NWP',
                                            'showarrow': False
                                         }, 
                                         {
                                            'x':15,
                                            'y':0.5,
                                            'xref':'x',
                                            'yref':'y',
                                            'text':'NYP',
                                            'showarrow': False
                                         }, 
                                         {
                                            'x':5,
                                            'y':5.5,
                                            'xref':'x',
                                            'yref':'y',
                                            'text':'ND',
                                            'showarrow': False
                                         },
                                         {
                                            'x':11,
                                            'y':-5,
                                            'xref':'x',
                                            'yref':'y',
                                            'text':'HOB',
                                            'showarrow': False
                                         }],

                         'shapes': [
                            # Newark to NY Penn
                            {
                                'type': 'line',
                                'x0': 0,
                                'y0': 0,
                                'x1': 15,
                                'y1': 0,
                                'line': {
                                    'color': 'rgb(255, 0, 0)',
                                    'width': 2,
                                },
                            },
                             # Onward to Newark Broad
                            {
                                'type': 'line',
                                'x0': 0,
                                'y0': 10,
                                'x1': 5,
                                'y1': 5,
                                'line': {
                                    'color': 'rgb(0,0,0)',
                                    'width': 2,
                                },
                            },
                            # Newark Broad St to Junction 1
                            {
                                'type': 'line',
                                'x0': 5,
                                'y0': 5,
                                'x1': 6.5,
                                'y1': 0,
                                'line': {
                                    'color': 'rgb(0,0,0)',
                                    'width': 2,
                                },
                            },
                            # Junction 2 to Hoboken 
                            {
                                'type': 'line',
                                'x0': 8,
                                'y0': 0,
                                'x1': 10,
                                'y1': -5,
                                'line': {
                                    'color': 'rgb(0,0,0)',
                                    'width': 2,
                                },
                            },
                            # Newark Broad Street
                             {
                                'type': 'circle',
                                'xref': 'x',
                                'yref': 'y',
                                'fillcolor': 'rgb(255,255,255)',
                                'x0': 4.75,
                                'y0': 4.75,
                                'x1': 5.25,
                                'y1': 5.25,
                                'line': {
                                    'color': 'rgb(0,0,0)',
                                },
                            },
                             # Newark Penn
                             {
                                'type': 'circle',
                                'xref': 'x',
                                'yref': 'y',
                                'fillcolor': 'rgb(255,255,255)',
                                'x0': 4.75,
                                'y0': -0.25,
                                'x1': 5.25,
                                'y1': 0.25,
                                'line': {
                                    'color': 'rgb(0,0,0)',
                                },
                            },
                             # Secaucus
                             {
                                'type': 'circle',
                                'xref': 'x',
                                'yref': 'y',
                                'fillcolor': 'rgb(255,255,255)',
                                'x0': 9.75,
                                'y0': -0.25,
                                'x1': 10.25,
                                'y1': 0.25,
                                'line': {
                                    'color': 'rgb(0,0,0)',
                                },
                            },
                             # Hoboken
                             {
                                'type': 'circle',
                                'xref': 'x',
                                'yref': 'y',
                                'fillcolor': 'rgb(255,255,255)',
                                'x0': 9.75,
                                'y0': -5.25,
                                'x1': 10.25,
                                'y1': -4.75,
                                'line': {
                                    'color': 'rgb(0,0,0)',
                                },
                            },
                            # New York Penn
                             {
                                'type': 'circle',
                                'xref': 'x',
                                'yref': 'y',
                                'fillcolor': 'rgb(255,255,255)',
                                'x0': 14.75,
                                'y0': 0.25,
                                'x1': 15.25,
                                'y1': -0.25,
                                'line': {
                                    'color': 'rgb(0,0,0)',
                                },
                            },
                             # NEC Beyond
                             {
                                'type': 'circle',
                                'xref': 'x',
                                'yref': 'y',
                                'fillcolor': 'rgb(255,255,255)',
                                'x0': -0.25,
                                'y0': 0.5,
                                'x1': 0.25,
                                'y1': -0.5,
                                'line': {
                                    'color': 'rgb(255,255,255)',
                                }
                            }
                         ]},

                        'frames': frames
                    }
    figure['layout']['sliders'] = {
        'args': [
            'transition', {
                'duration': 1000,
                'easing': 'linear'
            }
        ],
        'initialValue': times[0],
    #     'plotlycommand': 'animate',
        'values': times,
        'visible': True
    }

    figure['layout']['sliders'] = [sliders_dict]
    figure['layout']['updatemenus'] = [
        {
            'buttons': [
                {
                    'args': [None, {'frame': {'duration': 700, 'redraw': False},
                             'fromcurrent': True, 'transition': {'duration': 1000, 'easing': 'linear'}}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                    'transition': {'duration': 0}}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }
    ]
    plotly.offline.plot(figure)